In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Imports
!pip install rasterio
from sklearn.metrics import precision_recall_curve, auc
import os
import torch
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models.detection import maskrcnn_resnet50_fpn
import torch.optim as optim
import pandas as pd
import csv

# Custom Dataset Class
class CustomDataset(Dataset):
    def __init__(self, image_paths, mask_paths=None, transform=None, is_test=False):
        self.image_paths = image_paths
        self.mask_paths = mask_paths if mask_paths else None
        self.transform = transform
        self.is_test = is_test
        self.samples = [(img, mask) for img, mask in zip(image_paths, mask_paths)] if not is_test else [(img, None) for img in image_paths]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, mask_path = self.samples[idx]
        preprocessing = Preprocessing(image_path)
        image = preprocessing.preprocess_image()
        image = torch.tensor(image.transpose(2, 0, 1), dtype=torch.float32)

        if self.is_test:
            return image, {"image_id": torch.tensor([idx])}
        else:
            mask = plt.imread(mask_path)
            if mask.ndim == 3:
                mask = mask[:, :, 0]  # Convert to grayscale if necessary
            mask = torch.tensor(mask, dtype=torch.uint8)
            target = {
                "masks": mask.unsqueeze(0),
                "labels": torch.tensor([1], dtype=torch.int64),
                "image_id": torch.tensor([idx])
            }
            if self.transform:
                image = self.transform(image)
            return image, target

# Preprocessing Class
class Preprocessing:
    def __init__(self, image_path):
        self.image_path = image_path

    def load_bands(self):
        with rasterio.open(self.image_path) as src:
            blue = src.read(1)
            green = src.read(2)
            red = src.read(3)
            nir = src.read(4)
            swir1 = src.read(5)
            swir2 = src.read(6)
        return blue, green, red, nir, swir1, swir2

    def preprocess_image(self):
        blue, green, red, nir, swir1, swir2 = self.load_bands()
        ndvi = self.compute_ndvi(red, nir)
        evi = self.compute_evi(nir, red, blue)
        normalized_bands = [self.normalize_band(band) for band in [blue, green, red, nir, swir1, swir2]]
        image = np.stack(normalized_bands + [ndvi, evi], axis=-1)
        return image

    def normalize_band(self, band):
        return (band - np.min(band)) / (np.max(band) - np.min(band))

    def compute_ndvi(self, red, nir):
        return (nir - red) / (nir + red + 1e-8)  # Avoid division by zero

    def compute_evi(self, nir, red, blue, g=2.5, c1=6, c2=7.5, l=1):
        return np.clip(g * (nir - red) / (nir + c1 * red - c2 * blue + l + 1e-8), 0, 1)

# Mask R-CNN Model
class MaskRCNNModel:
    def __init__(self, device='cuda'):
        self.device = device
        self.model = maskrcnn_resnet50_fpn(weights="DEFAULT")
        self.model.backbone.body.conv1 = torch.nn.Conv2d(8, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.to(self.device)
        self.optimizer = optim.SGD([p for p in self.model.parameters() if p.requires_grad], lr=0.005, momentum=0.9, weight_decay=0.0005)

    def train(self, dataloader, num_epochs=10):
        self.model.train()
        for epoch in range(num_epochs):
            running_loss = 0.0
            for images, targets in dataloader:
                images = [image.to(self.device) for image in images]
                targets = [{k: v.to(self.device) for k, v in t.items()} for t in targets]
                self.optimizer.zero_grad()
                loss_dict = self.model(images, targets)
                loss = sum(loss for loss in loss_dict.values())
                running_loss += loss.item()
                loss.backward()
                self.optimizer.step()
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(dataloader):.4f}")

# Define output directory
OUTPUT_DIR = "'/kaggle/working/predictions/"
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_FILE = os.path.join(OUTPUT_DIR, "hls_submission.csv")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
train_csv = pd.read_csv("/kaggle/input/train-test/train_ds.csv")
test_csv = pd.read_csv("/kaggle/input/train-test/test_ds.csv")

train_image_paths = train_csv["Input"].tolist()
train_mask_paths = train_csv["Label"].tolist()
test_image_paths = test_csv["Input"].tolist()

# Data Transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
])

# Create Datasets and Dataloaders
train_dataset = CustomDataset(image_paths=train_image_paths, mask_paths=train_mask_paths, transform=transform)
test_dataset = CustomDataset(image_paths=test_image_paths, is_test=True, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Initialize and train the model
model = MaskRCNNModel(device=device)
model.train(train_dataloader, num_epochs=10)

# Evaluate the model and save predictions to CSV
def evaluate_and_save(model, dataloader, output_file):
    model.model.eval()
    results = []
    
    with torch.no_grad():
        for images, targets in dataloader:
            images = [image.to(model.device) for image in images]
            predictions = model.model(images)
            
            for idx, prediction in enumerate(predictions):
                image_id = targets[idx]["image_id"].item()
                if len(prediction["scores"]) > 0 and prediction["scores"][0].item() > 0.5:  # Seuil de confiance
                    label = 1  # Classe 1 (prédiction positive)
                else:
                    label = 0  # Classe 0 (prédiction négative)
                
                results.append([image_id, label])

    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["id","prediction"])
        writer.writerows(results)
    
    print(f"Predictions saved to {output_file}")

# Call evaluation function
evaluate_and_save(model, test_dataloader, OUTPUT_FILE)


ModuleNotFoundError: No module named 'rasterio'

In [ ]:
!pip install rasterio
import os
import torch
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models.segmentation import deeplabv3_resnet50
import torch.optim as optim
import torch.nn as nn
import pandas as pd

# Custom Dataset Class for 8-Channel Segmentation
class CustomSegmentationDataset(Dataset):
    def __init__(self, image_paths, mask_paths=None, transform=None, is_test=False):
        self.image_paths = image_paths
        self.mask_paths = mask_paths if not is_test else None
        self.transform = transform
        self.is_test = is_test
        self.samples = []

        if not is_test:
            for img_path, mask_path in zip(image_paths, mask_paths):
                if os.path.exists(img_path) and os.path.exists(mask_path):
                    self.samples.append((img_path, mask_path))
        else:
            for img_path in image_paths:
                if os.path.exists(img_path):
                    self.samples.append((img_path, None))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, mask_path = self.samples[idx]
        preprocessing = Preprocessing(image_path)
        image = preprocessing.preprocess_image()
        image = torch.tensor(image.transpose(2, 0, 1), dtype=torch.float32)  # (C, H, W)

        if self.is_test:
            return image, {"image_id": torch.tensor([idx])}
        else:
            mask = plt.imread(mask_path)

            # Convert mask to grayscale (if it has extra channels)
            if mask.ndim == 3:
                mask = mask[..., 0]  # Take only the first channel
                
            # Create a writable copy of the mask
            mask = mask.copy()  # Make the mask writable

            # Convert mask values to integers (0 for background, 1 for class)
            mask[mask == -9999] = 0  # Set -9999 as background
            mask[mask == 1] = 1       # Set 1 as the foreground class

            # Ensure it's a 2D tensor (H, W) and correct dtype
            mask = torch.tensor(mask, dtype=torch.long).squeeze(0)

            if self.transform:
                image = self.transform(image)
                
            print(mask.shape)  # Check the shape of the mask

            return image, mask  # Image (C, H, W), Mask (H, W)

# Preprocessing Class for 8 Channels
class Preprocessing:
    def __init__(self, image_path):
        self.image_path = image_path

    def load_bands(self):
        with rasterio.open(self.image_path) as src:
            blue = src.read(1)
            green = src.read(2)
            red = src.read(3)
            nir = src.read(4)
            swir1 = src.read(5)
            swir2 = src.read(6)
        return blue, green, red, nir, swir1, swir2

    def preprocess_image(self):
        blue, green, red, nir, swir1, swir2 = self.load_bands()
        ndvi = self.compute_ndvi(red, nir)
        evi = self.compute_evi(nir, red, blue)
        normalized_bands = [self.normalize_band(band) for band in [blue, green, red, nir, swir1, swir2]]
        image = np.stack(normalized_bands + [ndvi, evi], axis=-1)  # Stack 8 channels
        return image

    def normalize_band(self, band):
        return (band - np.min(band)) / (np.max(band) - np.min(band))

    def compute_ndvi(self, red, nir):
        return (nir - red) / (nir + red + 1e-6)

    def compute_evi(self, nir, red, blue, g=2.5, c1=6, c2=7.5, l=1):
        return np.clip(g * (nir - red) / (nir + c1 * red - c2 * blue + l), 0, 1)

# Training and Model Setup
class DeepLabV3Model:
    def __init__(self, num_classes=2, device='cuda'):
        self.device = device
        self.model = deeplabv3_resnet50(pretrained=True)

        # Modify input layer to accept 8 channels
        in_features = self.model.backbone.conv1.in_channels
        self.model.backbone.conv1 = nn.Conv2d(8, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Modify output layer for segmentation classes
        self.model.classifier[4] = nn.Conv2d(256, num_classes, kernel_size=1)

        self.model.to(self.device)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)

    def train(self, dataloader, num_epochs=10):
        self.model.train()
        for epoch in range(num_epochs):
            running_loss = 0.0
            for images, masks in dataloader:
                images, masks = images.to(self.device), masks.to(self.device)
                self.optimizer.zero_grad()
                outputs = self.model(images)['out']
                loss = self.criterion(outputs, masks)
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item()
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(dataloader):.4f}")

    def evaluate(self, dataloader):
        self.model.eval()
        iou_scores = []
        with torch.no_grad():
            for images, masks in dataloader:
                images, masks = images.to(self.device), masks.to(self.device)
                outputs = self.model(images)['out']
                preds = torch.argmax(outputs, dim=1)
                intersection = (preds & masks).float().sum()
                union = (preds | masks).float().sum()
                iou_scores.append(intersection / union)
        mean_iou = sum(iou_scores) / len(iou_scores)
        print(f"Mean IoU: {mean_iou:.4f}")

# Load dataset
train_csv = pd.read_csv("/kaggle/input/train-test/train_ds.csv")
test_csv = pd.read_csv("/kaggle/input/train-test/test_ds.csv")

train_image_paths = train_csv["Input"].tolist()
train_mask_paths = train_csv["Label"].tolist()
test_image_paths = test_csv["Input"].tolist()

# Data Transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
])
%cd /kaggle/input/geo-ai-hack

# Create Datasets and Dataloaders
train_dataset = CustomSegmentationDataset(image_paths=train_image_paths, mask_paths=train_mask_paths, transform=transform)
test_dataset = CustomSegmentationDataset(image_paths=test_image_paths, is_test=True, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Initialize and train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepLabV3Model(device=device)
model.train(train_dataloader, num_epochs=10)

# Evaluate the model
model.evaluate(test_dataloader)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 76.3 MB/s eta 0:00:00:00:0100:01
/kaggle/input/geo-ai-hack


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth
100%|██████████| 161M/161M [00:00<00:00, 221MB/s] 


torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])


<ipython-input-1-97249612a362>:97: RuntimeWarning: divide by zero encountered in divide
  return np.clip(g * (nir - red) / (nir + c1 * red - c2 * blue + l), 0, 1)


torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size(

In [ ]:
# Evaluate the model and save predictions to CSV
def evaluate_and_save(model, dataloader, output_file):
    model.model.eval()
    results = []
    
    with torch.no_grad():
        for images, targets in dataloader:
            images = [image.to(model.device) for image in images]
            predictions = model.model(images)
            
            for idx, prediction in enumerate(predictions):
                image_id = targets[idx]["image_id"].item()
                if len(prediction["scores"]) > 0 and prediction["scores"][0].item() > 0.5:  # Seuil de confiance
                    label = 1  # Classe 1 (prédiction positive)
                else:
                    label = 0  # Classe 0 (prédiction négative)
                
                results.append([image_id, label])

    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["id","prediction"])
        writer.writerows(results)
    
    print(f"Predictions saved to {output_file}")

# Call evaluation function
evaluate_and_save(model, test_dataloader, OUTPUT_FILE)
